<img src="../static/logopython.png" alt="Logo Python" style="width: 300px; display: inline"/>
<img src="../static/deimoslogo.png" alt="Logo Deimos" style="width: 300px; display: inline"/>

TODO: Buscar un ejemplo de QUEUE, por lo que dice aquí (sacado del libro Python Tutorial, tema de Brief Tour of the Standard Library, Multi-threading)

The preferred approach to task coordination is to concentrate all access to a resource in a single thread and then use the
Queue module to feed that thread with requests from other threads. Applications using Queue.Queue objects
for inter-thread communication and coordination are easier to design, more readable, and more reliable

# Clase 8: Asincronía en Python

Para empezar, tenemos que distinguir entre _multithreading_ y _multiprocessing_. Las dos imágenes siguientes muestran la diferencia

<img src="../static/multithreading.png" alt="Spyder" style="width: 350px;"/>
<img src="../static/multiprocessing.png" alt="Spyder" style="width: 350px;"/>

En Python podemos gestionar la asincronía con diferentes enfoques

## Multithreading

Mediante el uso del módulo de [threading](https://docs.python.org/3/library/threading.html?highlight=threading)

In [2]:
# %load thread_test.py
#!/usr/bin/env python3

from threading import Thread
from time import sleep

def snooze(i):
    print("Hilo {} durmiendo 5 segundos".format(i))
    sleep(5)
    print("Hilo {} despierto".format(i))

def main():
    threads = []

    for i in range(10):
        thread = Thread(target=snooze, args=(i, ))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()


Hilo 0 durmiendo 5 segundos
Hilo 1 durmiendo 5 segundos
Hilo 2 durmiendo 5 segundos
Hilo 3 durmiendo 5 segundos
Hilo 4 durmiendo 5 segundos
Hilo 5 durmiendo 5 segundos
Hilo 6 durmiendo 5 segundos
Hilo 7 durmiendo 5 segundos
Hilo 8 durmiendo 5 segundos
Hilo 9 durmiendo 5 segundos
Hilo 7 despiertoHilo 9 despierto
Hilo 8 despierto
Hilo 6 despierto
Hilo 5 despierto
Hilo 4 despierto
Hilo 3 despierto
Hilo 2 despierto
Hilo 1 despierto
Hilo 0 despierto



<div class="alert alert-warning">Es posible que te haya llamado la atención el bloque condicional desde donde se llama a main. Todas las instrucciones que van dentro de ese bloque se ejecutan solo cuando el script corre por si mismo (no cuando es importado desde otro script). Es una manera de asegurarse de que solo se ejecuta la función main cuando lancemos explicitamente el script.</div>

## Multiprocessing

Mediante el uso del módulo [multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

In [3]:
# %load multiprocessing_test.py
#!/usr/bin/env python3

from multiprocessing import Process
from time import sleep

def snooze(i):
    print("Proceso {} durmiendo 5 segundos".format(i))
    sleep(5)
    print("Proceso {} despierto".format(i))

def main():
    processes = []

    for i in range(10):
        process = Process(target=snooze, args=(i, ))
        process.start()
        processes.append(process)

    for process in processes:
        process.join()

if __name__ == "__main__":
    main()


## Multiprocessing con pool de procesos

Mediante el uso del objeto [Pool](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool) del módulo de multiprocessing, que permite la paralelización de varios procesos al mismo tiempo

In [ ]:
# %load multiprocessing_pool_test.py
#!/usr/bin/env python3

from multiprocessing import Pool
from time import sleep

def snooze(i):
    print("Proceso {} durmiendo 5 segundos".format(i))
    sleep(5)
    print("Proceso {} despierto".format(i))

def main():
    pool = Pool(processes=2) # numero de cpus
    results = []

    for i in range(10):
        result = pool.apply_async(snooze, (i, ))
        results.append(result)

    for result in results:
        result.get()

    pool.close()
    pool.join()

if __name__ == "__main__":
    main()


La diferencia entre los dos últimos casos es importante: 

* En el primer caso, tenemos varios procesos corriendo de __manera concurrente__ (10, en concreto). 
* En el segundo caso, tenemos varios procesos corriendo de __manera paralela__ (5, en concreto), pero seguimos teniendo 10 procesos concurrentes.

<div class="alert alert-info">__Concurrencia__ y __paralelismo__ no son lo mismo. Concurrencia es lo que se consigue con el _multitasking_, por ejemplo: una cpu que le va asignando _slots_ de tiempo a varias tareas, y éstas se van alternando. El paralelismo exige que haya físicamente corriendo dos o más tareas al mismo tiempo, por lo que se hace necesario más de una cpu (una por tarea).</div>

Si se ve más fácil, se puede recordar esto:

* _Paralelo_ es lo contrario de _serie_
* _Concurrente_ es lo contrario de _secuencial_

## Multithreading y multiprocessing

Se pueden combinar ambas técnicas a la vez

In [ ]:
# %load multiprocessing_threading_test.py
#!/usr/bin/env python3

from threading import Thread
from multiprocessing import Process
from time import sleep

def snooze(i):
    print("Hilo {} durmiendo 5 segundos".format(i))
    sleep(5)
    print("Hilo {} despierto".format(i))

def thread_snooze():
    threads = []

    for i in range(5):
        thread = Thread(target=snooze, args=(i, ))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

def main():
    processes = []

    for i in range(2):
        process = Process(target=thread_snooze)
        process.start()
        processes.append(process)

    for process in processes:
        process.join()

if __name__ == "__main__":
    main()


## Asyncio

Si estamos usando _Python 3.4 o superior_, podemos hacer uso de la librería [asyncio](https://docs.python.org/3/library/asyncio.html)

In [ ]:
# %load asyncio_test.py
#!/usr/bin/env python3

import asyncio

@asyncio.coroutine
def snooze(i):
    print("Corrutina {} durmiendo 5 segundos".format(i))
    yield from asyncio.sleep(5)
    print("Corrutina {} despierta".format(i))

def main():

    loop = asyncio.get_event_loop()
    tasks = []

    for i in range(10):
        task = asyncio.ensure_future(snooze(i))
        tasks.append(task)

    loop.run_until_complete(asyncio.wait(tasks))
    loop.close()

if __name__ == "__main__":
    main()


En este ejemplo vemos un concepto nuevo: [_coroutine_](https://docs.python.org/3/library/asyncio-task.html#coroutines). Las corrutinas son generadores a los que podemos enviar datos, no solo obtenerlos de ellos mediante yield

<div class="alert alert-warning">__Recuerda__: Solo podrás usar _asyncio_ si tienes Python 3.4 o superior. Caso contrario, tendrías que instalarlo a mano</div>

## Async/await

A partir de Python 3.5, podemos usar [async/await](https://docs.python.org/3/reference/expressions.html#await), que es más elegante

In [ ]:
# %load asyncawait_test.py
#!/usr/bin/env python3

import asyncio

async def snooze(i):
    print("Corrutina {} durmiendo 5 segundos".format(i))
    await asyncio.sleep(5)
    print("Corrutina {} despierta".format(i))


def main():

    loop = asyncio.get_event_loop()
    tasks = []

    for i in range(10):
        task = snooze(i)
        tasks.append(task)

    loop.run_until_complete(asyncio.wait(tasks))
    loop.close()


if __name__ == "__main__":
    main()


##### <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es"><img alt="Licencia Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Curso Python</span> por <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Jorge Arévalo</span> se distribuye bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es">Licencia Creative Commons Atribución 4.0 Internacional</a>.

---
_Las siguientes celdas contienen configuración del Notebook_

_Para visualizar y utlizar los enlaces a Twitter el notebook debe ejecutarse como [seguro](http://ipython.org/ipython-doc/dev/notebook/security.html)_

    File > Trusted Notebook

In [3]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_file = '../static/styles/style.css'
HTML(open(css_file, "r").read())